In [ ]:
import os

folders = ['ansm','dailymed','rxnorm','snomed','ocrx']
dirname = 'ocrx-owls/all-data/'

In [2]:
import pandas as pd
from os.path import join as pjoin
def clean_ocrx(df,filename):
    df = df.drop_duplicates().sort_values("sLabel")
    labels = df['sLabel'].str.upper().values
    codes = df['s'].values
    new_df =  pd.DataFrame([(str(val_code),str(val)) for val, val_code in zip(labels,codes)])
    new_df.columns = ['ocrx_code','ocrx_label']
    new_df = new_df.reset_index()
    new_df.to_csv(filename,index=False)

for filename in os.listdir('ocrx-owls/all-data/ocrx'):
    if filename.endswith(".csv"):
        fullpath = pjoin('ocrx-owls/all-data/ocrx',filename)
        df = pd.read_csv(fullpath,index_col=False)
        if 'ocrx_code' not in df.columns:
            clean_ocrx(df,fullpath)

In [3]:
from os.path import join as pjoin
import pandas as pd


def read_csv(fn):
    df = pd.read_csv(fn,index_col=False)
    for col in df.columns:
        if col.endswith("_label"):
            df[col] = df[col].str.upper()
        if col.startswith("Unnamed"):
            del df[col]
    return df.dropna().drop_duplicates()
def extract_from_fn(filename):
    without_ext = filename.split(".")[0]
    name = "_".join(without_ext.split("_")[1:-1])
    lang = without_ext.split("_")[-1]
#     if name == 'substance':
#         name = 'active_ingredient'
    return name, lang
all_files = {folder : {extract_from_fn(filename) : read_csv(pjoin(dirname,folder,filename)) for filename in os.listdir(pjoin(dirname,folder)) if filename.endswith(".csv")} for folder in folders}
# extract_from_fn("ansm_active_ingredient_en.csv")

In [4]:
all_files['ocrx'][('drug','fr')]

,ocrx_code,ocrx_label
0,http://www.ocrx.ca/OCRx/2196287095,(CHLORHYDRATE DE MÉTOCLOPRAMIDE MONOHYDRATÉ) 5...
1,http://www.ocrx.ca/OCRx/2402231607,(MALÉATE D'ACÉPROMAZINE ((MALÉATE D'ACÉPROMAZI...
3,http://www.ocrx.ca/OCRx/1000017403,(MALÉATE D'ACÉPROMAZINE ((MALÉATE D'ACÉPROMAZI...
4,http://www.ocrx.ca/OCRx/2561150065,(MALÉATE D'ACÉPROMAZINE ((MALÉATE D'ACÉPROMAZI...
5,http://www.ocrx.ca/OCRx/2320311874,(MALÉATE D'ACÉPROMAZINE ((MALÉATE D'ACÉPROMAZI...
...,...,...
293259,http://www.ocrx.ca/OCRx/2306181773,ÉTHINYLESTRADIOL 2.7 MG | ÉTONOGESTREL 11.7 MG...
293262,http://www.ocrx.ca/OCRx/1000021146,ÉTHOSUXIMIDE 250 MG/5 ML SOUS FORME DE SIROP P...
293278,http://www.ocrx.ca/OCRx/2331067000,ÉTONOGESTREL 68 MG SOUS FORME DE IMPLANT PAR V...
293296,http://www.ocrx.ca/OCRx/1000023041,ÉLAGOLIX (SODIUM DE ÉLAGOLIX) 150 MG SOUS FORM...


In [19]:
import os
DIR = 'match-ocrx-full'

from pytrie import Trie
def make_trie(df,prefix):
#     if len(df.columns) > 2:
#         import ipdb; ipdb.set_trace()
    code_col, label_col = f"{prefix}_code", f"{prefix}_label"
    return {row[label_col]: row[code_col] for _, row in df.iterrows()}

# def make_trie_ansm(ansm_def_df):
#     return {row['snomed_label']: row['snomed_code'] for _, row in ansm_def_df.iterrows()}

def get_prefix(trie,term):
    words = term.split(' ')
    for i in reversed(range(1,len(words)+1)):
        term = ' '.join(words[:i])
        if term in trie:
            return (term, trie[term])
    return None

def print_stats(df_merged,type_fn,dirname,name='rxnorm'):
    # total_match
    def get_count(df,col):
        return df[col].unique().shape[0]
    DIR = dirname
    prefix_code = f"{name}_code"
    ocrx_col = "ocrx_code"
    print("TOTAL MATCH:")
    total_df = df_merged.dropna()
    print(total_df.head())
    print(total_df.shape)
    total_df.to_csv(f"{DIR}/match_{type_fn}_total_match.csv")
    # left missing
    print("ONLY OCRX")
    only_ocrx = df_merged.loc[df_merged[prefix_code].isna()]
    print(only_ocrx.head())
    print(only_ocrx.shape)
    only_ocrx.to_csv(f"{DIR}/match_{type_fn}_only_ocrx.csv")
    # right missing
    print(f"ONLY {name.upper()}")
    only_ansm = df_merged.loc[df_merged['ocrx_code'].isna()]
    print(only_ansm.head())
    print(only_ansm.shape)
    stats = {
        'total' : total_df.shape[0], 'ocrx' : only_ocrx.shape[0], name: only_ansm.shape[0],
        'total_ocrx_unique' : get_count(total_df,ocrx_col) , f'total_{name}_unique' : get_count(total_df,prefix_code), 'ocrx_unique' : get_count(only_ocrx,ocrx_col), f"{name}_unique" : get_count(only_ansm,prefix_code)}
    only_ansm.to_csv(f"{DIR}/match_{type_fn}_only_{name}.csv")
    return stats

def perform_alignment(df1,df2,fn,dirname,prefix='rxnorm'):
    all_stats = dict()
    trie1 = make_trie(df1,'ocrx')
    trie2 = make_trie(df2,prefix)
    fullpath = f"{dirname}/{prefix}_{fn}_full.csv"
    df_merged = pd.merge(df1,df2,how="outer",left_on=['ocrx_label'], right_on=[f'{prefix}_label'])
#     import ipdb; ipdb.set_trace()
    df_merged_with_prefix = add_prefixes(df_merged,trie1,trie2,prefix=prefix)
#     all_stats[fn] = print_stats(df_merged_with_prefix,fn,dirname,prefix)
    df_merged_with_prefix.to_csv(fullpath)
    return all_stats

def add_prefixes(df,trie,trie_ansm,prefix='rxnorm'):
    prefix_code = f"{prefix}_code"
    prefix_label = f"{prefix}_label"
    def process_row(row,trie):
        if type(row['ocrx_code']) == str and type(row[prefix_code]) == str:
            return (row['ocrx_code'], row['ocrx_label'], 'EXACT')
        elif str(row['ocrx_code']) == 'nan':
            result = get_prefix(trie,row[prefix_label])
            if result is None:
                return (None, None, 'NONE')
            else:
                prefix_ocrx, ocrx_code = result
                return (ocrx_code, prefix_ocrx, 'PREFIX')
        else:
            return (row['ocrx_code'], row['ocrx_label'], 'NA')
    def process_row_ansm(row,trie):
        if type(row['ocrx_code']) == str and type(row[prefix_code]) == str:
            return (row[prefix_code], row[prefix_label], 'EXACT')
        elif str(row[prefix_code]) == 'nan':
            result = get_prefix(trie,row['ocrx_label'])
            if result is None:
                return (None, None, 'NONE')
            else:
                prefix_ansm, ansm_code = result
                return (ansm_code, prefix_ansm, 'PREFIX')
        else:
            return (row[prefix_code], row[prefix_label], 'NA')
        
            
#     import ipdb; ipdb.set_trace()
#     els = ['ocrx',prefix]
#     new_df = df.copy()
    match_list = []
    for _, row in df.iterrows():
        # check if ocrx needs match or other needs match.
        if type(row['ocrx_label']) == str and type(row[prefix_label]) != str:
             # we don't care about this.
            continue
        if type(row[prefix_label]) == str and type(row['ocrx_label']) != str:
            result = get_prefix(trie,row[prefix_label])
            if result is None:
                match_list.append((row[prefix_code],row[prefix_label],None,None,'NONE'))
            else:
                matched_name, matched_code = result
                match_list.append((row[prefix_code],row[prefix_label],matched_code, matched_name,'PREFIX'))
        if type(row[prefix_label]) == str and type(row['ocrx_label']) == str:
            match_list.append((row[prefix_code],row[prefix_label],row['ocrx_code'],row['ocrx_label'],'EXACT'))
    new_df = pd.DataFrame(match_list)
    new_df.columns = [prefix_code,prefix_label,'ocrx_code','ocrx_label','match_type']
    return new_df

In [20]:
all_files.keys()
matched = ['ansm','dailymed','rxnorm','snomed']


In [21]:
hi = dict()
hi.update({"hi" : "ho"})

In [22]:
hi

{'hi': 'ho'}

In [23]:
import re
def clean_form_snomed(raw_form_df):
    mapping = {
        "EYE" : "OPHTALMIC",
        "EAR" : "OTIC",
        "NOSE" : "NASAL"
    }
    def sub_form_snomed_helper(form):
        return ' '.join([mapping.get(el,el) for el in form.split(" ")])
    form_df = raw_form_df.copy()
    form_df['old_snomed_label'] = form_df['snomed_label'].copy()
    form_df['snomed_label'] = [sub_form_snomed_helper(el.replace(" DOSE FORM","")) for el in form_df['old_snomed_label'].values]
    return form_df

# def clean_drug_snomed(raw_drug_df):
#     ocrx_patt = re.compile('AS (.*) IN (.*)')
#     def clean_drug_snomed_helper(drug):
        

def clean_form_ocrx(raw_form_df):
    def clean_form_ocrx_helper(el):
        # removes the word in and inverts it
        if " IN " in el:
            splitted = el.split(" IN ")
    #             if len(splitted) > 2:
    #                 import ipdb; ipdb.set_trace()
            start, last = splitted[:2]
            return last + " " + start
        else:
            return el

    form_df = raw_form_df.copy()
    #     import ipdb; ipdb.set_trace()
    form_df['old_ocrx_label'] = form_df['ocrx_label'].copy()
    form_df['ocrx_label'] = [clean_form_ocrx_helper(el) for el in form_df['old_ocrx_label'].values]
    return form_df

helper_fns = {
    'snomed' : {('form','en') : {'snomed' : clean_form_snomed, 'ocrx' : clean_form_ocrx}},
}

In [24]:
%pdb on

Automatic pdb calling has been turned ON


In [25]:
if not os.path.exists("export-match"):
    os.mkdir("export-match")
all_stats = dict()
for other_prefix in matched:
    prefix_stats = dict()
    for id_tup, other_df in all_files[other_prefix].items():
        if id_tup[0] == 'component' or id_tup[0] == 'drug':
            continue
#         import ipdb; ipdb.set_trace()
#         if id_tup != ('drug','fr'):
#             continue
#         import ipdb; ipdb.set_trace()
        if id_tup not in all_files['ocrx']:
            continue
        name,lang = id_tup
        ocrx_df = all_files['ocrx'][id_tup]
        output_dir = f"export-match/{other_prefix}"
        if not os.path.exists(output_dir):
            os.mkdir(output_dir)
        if ocrx_df is None:
            import ipdb; ipdb.set_trace() 
#             print(ocrx_df.shape  [0])
#             continue
        if id_tup in helper_fns.get(other_prefix,dict()):
            ocrx_df = helper_fns.get(other_prefix)[id_tup]['ocrx'](ocrx_df)
            other_df = helper_fns.get(other_prefix)[id_tup][other_prefix](other_df)
#             import ipdb; ipdb.set_trace()
        elif name == 'form':
            ocrx_df = clean_form_ocrx(ocrx_df)
#         import ipdb; ipdb.set_trace()
        stats = perform_alignment(ocrx_df,other_df,name,output_dir,prefix=other_prefix)
        prefix_stats.update(stats)
    all_stats[other_prefix] = prefix_stats

In [ ]:
!code .

In [26]:
from pprint import pprint
pprint(all_stats)

{'ansm': {}, 'dailymed': {}, 'rxnorm': {}, 'snomed': {}}


```json
{'ansm': {'active_ingredient': {'ansm': 1753, 'ocrx': 315431, 'total': 120077},
          'drug': {'ansm': 15695, 'ocrx': 293298, 'total': 0},
          'form': {'ansm': 74, 'ocrx': 247, 'total': 600},
          'roa': {'ansm': 40, 'ocrx': 71, 'total': 27}},
 'dailymed': {'active_ingredient': {'dailymed': 554,
                                    'ocrx': 1199306,
                                    'total': 222566},
              'drug': {'dailymed': 45828, 'ocrx': 578640, 'total': 133066},
              'form': {'dailymed': 80, 'ocrx': 798, 'total': 50},
              'roa': {'dailymed': 30, 'ocrx': 50, 'total': 48}},
 'rxnorm': {'active_ingredient': {'ocrx': 1091070,
                                  'rxnorm': 10878,
                                  'total': 330801},
            'component': {'ocrx': 1479332, 'rxnorm': 11613, 'total': 14270},
            'drug': {'ocrx': 711706, 'rxnorm': 17274, 'total': 0},
            'form': {'ocrx': 806, 'rxnorm': 5, 'total': 41},
            'roa': {'ocrx': 77, 'rxnorm': 10, 'total': 21}},
 'snomed': {'active_ingredient': {'ocrx': 1050462,
                                  'snomed': 21076,
                                  'total': 371409},
            'drug': {'ocrx': 711706, 'snomed': 7894, 'total': 0},
            'form': {'ocrx': 606, 'snomed': 263, 'total': 61},
            'roa': {'ocrx': 80, 'snomed': 14, 'total': 18}}}
```

# Part 2

In [5]:
1+1

2

In [27]:
def count_for_prefix(df,prefix,match_type):
    match_type_col_name = f"{prefix}_match_type"
    label_name = f"{prefix}_label"
    
    
    sub_df = df.dropna(subset=['ocrx_label',label_name])
#     import ipdb; ipdb.set_trace()
    counts = [el.shape[0] for _, el in sub_df.groupby(label_name)]
    count_1 = sum([1 if count == 1 else 0 for count in counts])
    count_n = sum([1 if count > 1 else 0 for count in counts])
    return {'1-1' : count_1, '1-n' : count_n}
def analyze_df(df,prefix):
    stats = dict()
    ocrx_match_type = "ocrx_match_type"
    prefix_match_type = f"{prefix}_match_type"
    match_types = ['EXACT','PREFIX']
    prefix_options = ['ocrx',prefix]
    import ipdb; ipdb.set_trace()
    for prefix_el in prefix_options:
        for match_type_el in match_types:
            stats[prefix+'-'+match_type_el] = count_for_prefix(df,prefix_el,match_type_el)
    return stats

In [42]:
def calculate_stats(df,prefix,match_type = None):
    # removes na match for alternative spelling of labels
    label_col = f'{prefix}_label'
    code_col = f'{prefix}_code'

    not_na_df = df.dropna(subset=['ocrx_label',label_col])
    if match_type is not None:
        not_na_df = not_na_df.loc[not_na_df['match_type'] == match_type]
#     if match_type is not None:
#         import ipdb; ipdb.set_trace()
#         not_na_df = not_na_df.loc[not_na_df['ocrx_match_type'] == match_type]
    counts = [len(el['ocrx_code'].unique()) for _, el in not_na_df.groupby(code_col)]
    count_1 = sum([1 if count == 1 else 0 for count in counts])
    count_n = sum([1 if count > 1 else 0 for count in counts])
    
    unique_other_ids = set(not_na_df[code_col].values)
    unique_ocrx_ids = set(not_na_df['ocrx_code'].values)
    na_df = df[df['ocrx_label'].isna()]
    filtered = [row[code_col] not in unique_other_ids for _, row in na_df.iterrows()]
    count_0 = len(df.loc[df['match_type'] == 'NONE'][code_col].unique())
#     assert count_0 == len(df[code_col].unique()) - count_1 - count_n
    if len(filtered) != len(na_df):
        import ipdb; ipdb.set_trace()
    return {'a1-n' : count_n, 'b1-1' : count_1,'c1-0' : count_0 }
    

In [43]:
import os
from os.path import join as pjoin

In [49]:
%pdb on
import pandas as pd
folders = ['ansm','dailymed','snomed','rxnorm']
dirname = 'export-match'
dfs = dict()
stats = dict()
for folder in folders:
    fullfolder = pjoin(dirname,folder)
    for filename in os.listdir(fullfolder):
        if filename.endswith("_full.csv") and 'drug' not in filename and 'component' not in filename:
            name = filename.split("_")[0]
            if name not in dfs:
                dfs[name] = dict()
            tag = "_".join(filename.split("_")[1:-1])
            fullfile = pjoin(fullfolder,filename)
            test_df = pd.read_csv(fullfile,index_col=False)
            mode = ['PREFIX','EXACT']
            for mode_el in mode:
                single_stats = calculate_stats(test_df,folder,mode_el)
                stats[filename+'-'+mode_el] = single_stats
            stats[filename] = calculate_stats(test_df,folder)
#             no_match = df[df['ocrx_code'].isna()][folder+"_code"].unique().shape[0]
            dfs[name][tag] = test_df
                

Automatic pdb calling has been turned ON


In [50]:
stats.keys()

dict_keys(['ansm_strength_full.csv-PREFIX', 'ansm_strength_full.csv-EXACT', 'ansm_strength_full.csv', 'ansm_form_full.csv-PREFIX', 'ansm_form_full.csv-EXACT', 'ansm_form_full.csv', 'ansm_roa_full.csv-PREFIX', 'ansm_roa_full.csv-EXACT', 'ansm_roa_full.csv', 'ansm_active_ingredient_full.csv-PREFIX', 'ansm_active_ingredient_full.csv-EXACT', 'ansm_active_ingredient_full.csv', 'dailymed_roa_full.csv-PREFIX', 'dailymed_roa_full.csv-EXACT', 'dailymed_roa_full.csv', 'dailymed_strength_full.csv-PREFIX', 'dailymed_strength_full.csv-EXACT', 'dailymed_strength_full.csv', 'dailymed_form_full.csv-PREFIX', 'dailymed_form_full.csv-EXACT', 'dailymed_form_full.csv', 'dailymed_active_ingredient_full.csv-PREFIX', 'dailymed_active_ingredient_full.csv-EXACT', 'dailymed_active_ingredient_full.csv', 'snomed_roa_full.csv-PREFIX', 'snomed_roa_full.csv-EXACT', 'snomed_roa_full.csv', 'snomed_form_full.csv-PREFIX', 'snomed_form_full.csv-EXACT', 'snomed_form_full.csv', 'snomed_active_ingredient_full.csv-PREFIX', 's

In [51]:
stats

{'ansm_strength_full.csv-PREFIX': {'a1-n': 0, 'b1-1': 1294, 'c1-0': 5648},
 'ansm_strength_full.csv-EXACT': {'a1-n': 0, 'b1-1': 0, 'c1-0': 5648},
 'ansm_strength_full.csv': {'a1-n': 0, 'b1-1': 1294, 'c1-0': 5648},
 'ansm_form_full.csv-PREFIX': {'a1-n': 0, 'b1-1': 298, 'c1-0': 74},
 'ansm_form_full.csv-EXACT': {'a1-n': 0, 'b1-1': 25, 'c1-0': 74},
 'ansm_form_full.csv': {'a1-n': 0, 'b1-1': 323, 'c1-0': 74},
 'ansm_roa_full.csv-PREFIX': {'a1-n': 0, 'b1-1': 1, 'c1-0': 40},
 'ansm_roa_full.csv-EXACT': {'a1-n': 0, 'b1-1': 27, 'c1-0': 40},
 'ansm_roa_full.csv': {'a1-n': 0, 'b1-1': 28, 'c1-0': 40},
 'ansm_active_ingredient_full.csv-PREFIX': {'a1-n': 0,
  'b1-1': 1241,
  'c1-0': 1532},
 'ansm_active_ingredient_full.csv-EXACT': {'a1-n': 7,
  'b1-1': 1136,
  'c1-0': 1532},
 'ansm_active_ingredient_full.csv': {'a1-n': 7, 'b1-1': 2377, 'c1-0': 1532},
 'dailymed_roa_full.csv-PREFIX': {'a1-n': 0, 'b1-1': 0, 'c1-0': 30},
 'dailymed_roa_full.csv-EXACT': {'a1-n': 0, 'b1-1': 42, 'c1-0': 30},
 'dailymed_r

In [53]:
from pprint import pprint
# pprint(stats)

tables = ['rxnorm','snomed','ansm']
terms = ['active_ingredient','strength','form','roa']
for table in tables:
    print(f"NOW DOING {table}")
    for term in terms:
       
        filename = f"{table}_{term}_full.csv"
        if filename not in stats:
            continue
        print(term  +  ": " + str(stats[filename]))

NOW DOING rxnorm
active_ingredient: {'a1-n': 6, 'b1-1': 6428, 'c1-0': 10878}
form: {'a1-n': 0, 'b1-1': 50, 'c1-0': 5}
roa: {'a1-n': 0, 'b1-1': 17, 'c1-0': 10}
NOW DOING snomed
active_ingredient: {'a1-n': 6, 'b1-1': 6109, 'c1-0': 21076}
form: {'a1-n': 0, 'b1-1': 183, 'c1-0': 263}
roa: {'a1-n': 0, 'b1-1': 17, 'c1-0': 14}
NOW DOING ansm
active_ingredient: {'a1-n': 7, 'b1-1': 2377, 'c1-0': 1532}
strength: {'a1-n': 0, 'b1-1': 1294, 'c1-0': 5648}
form: {'a1-n': 0, 'b1-1': 323, 'c1-0': 74}
roa: {'a1-n': 0, 'b1-1': 28, 'c1-0': 40}


In [52]:
from pprint import pprint
# pprint(stats)

tables = ['rxnorm','snomed','ansm']
terms = ['active_ingredient','strength','form','roa']
for table in tables:
    print(f"NOW DOING {table}")
    for term in terms:
        for mode in ['EXACT','PREFIX']:
            filename = f"{table}_{term}_full.csv-{mode}"
            if filename not in stats:
                continue
            print(term + '-'+ mode +  ": " + str(stats[filename]))

NOW DOING rxnorm
active_ingredient-EXACT: {'a1-n': 6, 'b1-1': 3591, 'c1-0': 10878}
active_ingredient-PREFIX: {'a1-n': 0, 'b1-1': 2837, 'c1-0': 10878}
form-EXACT: {'a1-n': 0, 'b1-1': 38, 'c1-0': 5}
form-PREFIX: {'a1-n': 0, 'b1-1': 12, 'c1-0': 5}
roa-EXACT: {'a1-n': 0, 'b1-1': 17, 'c1-0': 10}
roa-PREFIX: {'a1-n': 0, 'b1-1': 0, 'c1-0': 10}
NOW DOING snomed
active_ingredient-EXACT: {'a1-n': 6, 'b1-1': 3283, 'c1-0': 21076}
active_ingredient-PREFIX: {'a1-n': 0, 'b1-1': 2826, 'c1-0': 21076}
form-EXACT: {'a1-n': 0, 'b1-1': 61, 'c1-0': 263}
form-PREFIX: {'a1-n': 0, 'b1-1': 122, 'c1-0': 263}
roa-EXACT: {'a1-n': 0, 'b1-1': 17, 'c1-0': 14}
roa-PREFIX: {'a1-n': 0, 'b1-1': 0, 'c1-0': 14}
NOW DOING ansm
active_ingredient-EXACT: {'a1-n': 7, 'b1-1': 1136, 'c1-0': 1532}
active_ingredient-PREFIX: {'a1-n': 0, 'b1-1': 1241, 'c1-0': 1532}
strength-EXACT: {'a1-n': 0, 'b1-1': 0, 'c1-0': 5648}
strength-PREFIX: {'a1-n': 0, 'b1-1': 1294, 'c1-0': 5648}
form-EXACT: {'a1-n': 0, 'b1-1': 25, 'c1-0': 74}
form-PREFIX: 

In [ ]:
dfs

In [ ]:
analyze_df(dfs['rxnorm']['active_ingredient'],'rxnorm')

In [ ]:
dfs['rxnorm']['active_ingredient']['ocrx_match_type'].value_counts()

In [ ]:
test_df = dfs['rxnorm']['active_ingredient']
test_df.loc[test_df['rxnorm_match_type'] == 'PREFIX']
# dfs['rxnorm']['active_ingredient']['rxnorm_match_type'].value_counts()